#변수 정의

In [19]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/model/KoBERT_emotion_finetuned_data_kw.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 52.5 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=57a758c33646402d7137a6fac63cac7628b8555365b95075caccf1d563fbc0fc
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 769 kB 4.1 MB/s 
     |████████████████████████████████| 895 kB 40.1 MB/s 
     |████████████████████████████████| 3.0 MB 32.7 MB/s 


#KoBERT 다운로드#

In [2]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-6zut3enr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-6zut3enr
     |████████████████████████████████| 4.9 MB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 34.0 MB/s 
     |████████████████████████████████| 61 kB 498 kB/s 
     |████████████████████████████████| 3.3 MB 37.9 MB/s 
     |████████████████████████████████| 596 kB 58.5 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=a6aec15c4e197c50fcc40249fab2f9b2a48f7463d8199867d2196dea3009c092
  Stored in directory: /tmp/pip-ephem-wheel-cache-ypg3eujr/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
#GPU 사용
device = torch.device("cuda:0")

In [6]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#데이터셋 전처리#

In [17]:
#구글드라이브 연동
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [20]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep answer').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1RBIKbJaDHdamnXBjtqOvW1Mq0AELWzgD/SEP531/kor_kw_labeled/train
/content
popd -> /content


In [21]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep answer').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/.shortcut-targets-by-id/1RBIKbJaDHdamnXBjtqOvW1Mq0AELWzgD/SEP531/kor_kw_labeled/valid
/content
popd -> /content


In [22]:
train_json_list[0]

'DGIN21910194.json.answer'

In [23]:
!cat $TRAIN_DIR/{train_json_list[0]}

그래 너 땜에 꿈자리 사나왔다 이눔아.	그랴 너 땜에 꿈자리 사나왔다 이눔아.	4
이 날라리야 어디서 나쁜 것만 닮아가지구.	이 날라리야 워드메서 나쁜 것만 닮아가지구.	6
결혼도 하기 전에 속 썩여?	겨론두 하기 전에 속 썩여?	0
어떻게 아셨어요?	우타 아셨어요?	1
어떻게 알건.	우타 알건.	4
아유 남자가 술집 갈 수 있죠.	아유 남자가 술집 갈 수 있죠.	1
그냥 술집이야?	처레미 술집이야?	4
룸싸롱 단골이라매?	룸싸롱 단골이라매?	1
뭐 단골이예요.	뭐 단골이예요.	4
친구들 따라서 몇번.	친구들 따러서 몇번.	4
단 한번이래두.	단 한번이래두.	2
벌써부터 그런데나 출입하구 뭐하는 짓이야.	발싸부터 그런데나 출입하구 뭐하는 짓이야.	6
아유 내가 호랑이 굴엘 들왔어 나만 잡어.	아유 내가 호라이 굴엘 들왔어 나만 잡어.	6
남 탓하지 마 니 허물이야 니 아버지는 한 번이나 어디 그런데서 다니시니?	남 탓하지 마 니 허물이야 니 아부지는 한 번이나 워드메 그런데서 다니시니?	6
아버지야 교육자시니까요.	아부지야 교육자시니까요.	4
교육자 하구 일반 직업 하구 뭐가 달러.	교육자 하구 일반 직업 하구 뭐가 달러.	4
교육자 가기 -북적- 부적절한 곳은 일반인두 가면 안되지.	교육자 가기 -북적- 부적절한 곳은 일반인두 가면 안되지.	4
할머니.	할멍이.	4
엄마.	어멍이.	5
고모.	아짐.	1
남자루 태어났으면 여기 세 분두 마찬가지예요.	남자루 태어났으면 여개 세 분두 마찬가지예요.	0
똑같이 그런데 가서 술마시구 바람두 피구 했을 거예요.	똑같이 그런데 가서 술마시구 바람두 피구 했을 거예요.	3
저 저.	저 저.	4
여자들은 상황이 안돼서 못 가는 거지 너무 죄인 취급 하지 마세요.	예자들은 상황이 안돼서 못 가는 거지 너머 죄인 취급 하지 마세요.	3
&name1& 보는데서 어떻게 애인 타령하구 끌어안구 그래.	&name1& 보는데서 우타 애인 타령하구 끌어안구 그래.	1
언제 끌어안아요?	운제 끌어안아요?	1
왜 잡아 떼 &name1&가 봤

In [24]:
import pandas as pd

chatbot_data = pd.read_csv('%s/%s' % (TRAIN_DIR, train_json_list[0]), sep='\t', header=None)

chatbot_data

,0,1,2
0,그래 너 땜에 꿈자리 사나왔다 이눔아.,그랴 너 땜에 꿈자리 사나왔다 이눔아.,4
1,이 날라리야 어디서 나쁜 것만 닮아가지구.,이 날라리야 워드메서 나쁜 것만 닮아가지구.,6
2,결혼도 하기 전에 속 썩여?,겨론두 하기 전에 속 썩여?,0
3,어떻게 아셨어요?,우타 아셨어요?,1
4,어떻게 알건.,우타 알건.,4
...,...,...,...
249,사회서나 가정에서나 샐러리맨들은 거의 굶구 나온대잖아.,사회서나 가정에서나 샐러리맨들은 거우 굶구 나온대잖아.,3
250,회사 나와서 컵라면 김밥 사먹느라 정신없구.,회사 나와서 컵라면 김밥 사먹느라 정신없구.,3
251,에이그 한심들 해.,에이그 한심들 해.,6
252,잠 펴자면서 서방 굶겨 내보내는 여편네야들 그런다구 빈속으루 쫓겨나오는 위인들이나.,잠 펴자면서 서방 굶겨 내보내는 여편네야들 그른다구 빈속으루 쫓겨나오는 위인들이나.,6


In [25]:
def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[1], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [26]:
dataset_train = GenDataset(TRAIN_DIR, train_json_list)

DGIN21910194.json.answer

DGIN21910221.json.answer

DGIN21910222.json.answer

DGTS20000188.json.answer

DGTS20000191.json.answer

DGTS20000199.json.answer

DGTS20000200.json.answer

DGTS20000209.json.answer

DGTS20000215.json.answer

DGTS20000221.json.answer

DGTS20000222.json.answer

DGTS20000231.json.answer

DGTS20000232.json.answer

DGTS20000233.json.answer

DGTS20000234.json.answer

DGTS20000235.json.answer

DGTS20000238.json.answer

DGTS20000239.json.answer

DGTS20000240.json.answer

DGTS20000242.json.answer

DGTS20000243.json.answer

DGTS20000244.json.answer

DGTS20000245.json.answer

DGTS20000246.json.answer

DGTS20000247.json.answer

DGTS20000248.json.answer

DGTS20000249.json.answer

DGTS20000250.json.answer

DGTS20000251.json.answer

DGTS20000252.json.answer

DGTS20000253.json.answer

DGTS20000254.json.answer

DGTS20000255.json.answer

DGTS20000256.json.answer

DGTS20000257.json.answer

DGTS20000258.json.answer

DGTS20000259.json.answer

DGTS20000260.json.answer

DGTS20000262

In [27]:
dataset_valid = GenDataset(VALID_DIR, valid_json_list)

DGIN21800347.json.answer

DGIN21800351.json.answer

DGIN21800352.json.answer

DGIN21800353.json.answer

DGIN21800354.json.answer

DGIN21800382.json.answer

DGIN21800392.json.answer

DGIN21800405.json.answer

DGIN21800434.json.answer

DGIN21800438.json.answer

DGIN21800447.json.answer

DGIN21800454.json.answer

DGIN21800457.json.answer

DGIN21800459.json.answer

DGIN21800460.json.answer

DGIN21800461.json.answer

DGIN21800470.json.answer

DGIN21800481.json.answer

DGIN21800487.json.answer

DGIN21800489.json.answer

DGIN21800490.json.answer

DGIN21800510.json.answer

DGIN21800512.json.answer

DGIN21800515.json.answer

DGIN21800519.json.answer

DGIN21800520.json.answer

DGIN21800521.json.answer

DGIN21800522.json.answer

DGIN21800523.json.answer

DGIN21800524.json.answer

DGIN21800527.json.answer

DGIN21800530.json.answer

DGIN21800532.json.answer

DGIN21800533.json.answer

DGIN21800534.json.answer

DGIN21800535.json.answer

DGIN21800538.json.answer

DGIN21800542.json.answer

DGIN21800547

In [28]:
print(len(dataset_train))
print(len(dataset_valid))
dataset_train = dataset_train[0:45000]
dataset_valid = dataset_valid[0:15000]

print("slice")
print(len(dataset_train))
print(len(dataset_valid))

94485
26727
slice
45000
15000


#KoBERT 입력 데이터로 변환#

In [29]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [30]:
# Setting parameters
max_len = 64
batch_size = 64 # origin : 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [31]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False)

using cached model


In [32]:
# 첫번째는 패딩된 시퀀스, 두번째는 길이와 티입, 세번째는 어텐션 마스크 시퀀스
data_train[0]

(array([   2, 1185, 6033, 1457,  517,    0, 6896, 1356, 7159, 2573, 5655,
        6990, 3647, 5747, 6797,  517,   54,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 4)

In [34]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#KoBERT 학습모델 만들기#

In [33]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [35]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_added.pt'), strict=False)
model.eval()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

#KoBERT 모델 학습시키기#

In [36]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.2656532526016235 train acc 0.625
epoch 1 batch id 201 loss 0.6334914565086365 train acc 0.699160447761194
epoch 1 batch id 401 loss 1.0781992673873901 train acc 0.6962281795511222
epoch 1 batch id 601 loss 1.06773841381073 train acc 0.6844061980033278
epoch 1 train acc 0.6903852982954546


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6776374113475176


  0%|          | 0/704 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2416927069425583 train acc 0.921875
epoch 5 batch id 201 loss 0.005438814405351877 train acc 0.9761349502487562
epoch 5 batch id 401 loss 0.0660456195473671 train acc 0.9745168329177057
epoch 5 batch id 601 loss 0.12126734852790833 train acc 0.9716878119800333
epoch 5 train acc 0.9723677201704546


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 5 test acc 0.6813608156028368


In [37]:
torch.save(model.state_dict(), SAVEPOINT_PATH)